In [1]:
import sqlite3
import json
from os import getenv
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pytesseract
from PIL import Image
import cv2
import numpy as np
from re import split
import csv

In [38]:
con = sqlite3.connect("mangakakalot.db")
cur = con.cursor()

for row in cur.execute("SELECT * FROM Manga"):
    title, url, current, latest = row

    current = current.split(":")[0].split(" ")[-1]
    if current == "" or current == " ":
        current = 0
    elif "." in current:
        current = float(current)
    else:
        current = int(current)

    latest = latest.split(":")[0].split(" ")[-1]
    if latest == "" or latest == " ":
        latest = 0
    elif "." in latest:
        latest = float(latest)
    else:
        latest = int(latest)

1 559
9 25
26 72
60 73
43 165
24 26
13.2 14
144 145
163 175
9 155
2 167
121 123
73 75
16 17.2
35 140
23 45
152 210
14 18
37 38
12 13
109 115
3 11
42 50
44 46
73 124
5 34.3
13 14
12 23
9 15
79 80
91 105
64 66
283 476.1
9 10.5
46 64
63 64
11 13
17.2 18.2
27 28
21 24
27.2 36
61 116
88 166
47 49
13 19
49 79
22 25
52 69
52 54
22.2 30
37 46
46 116
0 59
190 327
54 66
3.1 3.2
74 74.5
40 50
8 23.7
20 27
106 106.3
6.1 7.2
48 72
46 269
49 93
0 0
8.1 8.3
12 29.2
47 69
32 80
68 68
30 30
28 28
52 52
12 12
2.3 2.3
11 11
100 100
13 13
15 15
15 15
12.2 12.2
6 6
153 153
16 16
101 101
1 1
172 172
4 4
46 46
166 166


In [37]:
test = "132"
if "." in test:
    print(True)

In [54]:
def func(line):
    line = line.strip('"')
    curr_chap, latest_chap = "", ""

    if line[0] == "C": # it starts with "Chapter"
        line = line.split("Chapter")[1:]
        first = line[0].split("Vol")

        if len(first) == 1:
            curr_chap = "Chapter" + line[0][:-1]
            latest_chap = "Chapter " + line[1].strip()
        elif len(first) == 2:
            curr_chap = "Chapter" + first[0][:-1]
            latest_chap = "Vol" + first[1] + "Chapter " + line[-1].strip()

    else: # it starts with "Vol"
        line = line.split("Vol")[1:]

        if len(line) == 1: # the latest chapter starts with "Chapter"
            line = line[0].rsplit(",Chapter", 1)
            curr_chap = "Vol" + line[0]
            latest_chap = "Chapter " + line[-1].strip()
        elif len(line) == 2: # the latest chapter starts with "Vol"
            curr_chap = "Vol" + line[0][:-1]
            latest_chap = "Vol" + line[1].strip()

        # print(f"{curr_chap} --- {latest_chap}")

    return curr_chap, latest_chap

one = "Vol.5 Chapter 43: Cheers,Vol.15 Chapter 165: Takehi Gouda's Ability\n"
two = "Chapter 3,Chapter 11\n"
three = "Chapter 14,Vol.4 Chapter 18\n"
four = "Vol.4 Chapter 14,Chapter 18: Something else is going on\n"

print(func(four))

('Vol.4 Chapter 14', 'Chapter 18: Something else is going on')


In [55]:
titles = []
urls = []
current = []
latest = []
full = []

with open("tempdb.csv", "r", encoding="utf-8") as file:
    csvwriter = csv.reader(file)
    for i, lines in enumerate(file):
        full.append(lines)
        if i == 0:
            continue

        lines = split("https", lines)
        titles.append(lines[0][:-1].strip('"'))

        lines = lines[1].split(",", maxsplit=1)
        urls.append(f"https{lines[0]}")
        
        curr_chap, latest_chap = func(lines[1])
        current.append(curr_chap)
        latest.append(latest_chap)

In [27]:
driver = webdriver.Chrome("..\..\web_scraping\chromedriver-win64\chromedriver.exe")
driver.get("https://comick.io/home")

In [57]:
title = titles[0]
search_form = driver.find_element_by_xpath("//form[contains(@itemtype, 'http://schema.org/SearchAction')]")
search_bar = search_form.find_element_by_xpath(".//input")
search_button = search_form.find_element_by_xpath(".//button")
search_bar.send_keys(title)
search_button.click()

In [71]:
results_name = driver.find_elements_by_xpath(".//div[contains(@class, 'tt')]")[:5]
results_img = driver.find_elements_by_xpath(".//div[contains(@class, 'limit')]/img")
results_img = [r.get_attribute("src") for r in results_img]

https://img.asuracomics.com/unsafe/fit-in/330x450/filters:format(webp)/https://asuratoon.com/wp-content/uploads/2024/05/MargravesBastardSonwasTheEmperorCover02CUnetauto_scaleLevel3width-2000.jpg
https://img.asuracomics.com/unsafe/fit-in/330x450/filters:format(webp)/https://asuratoon.com/wp-content/uploads/2021/07/8thclass.jpg
https://img.asuracomics.com/unsafe/fit-in/330x450/filters:format(webp)/https://asuratoon.com/wp-content/uploads/2021/10/cover.png


In [75]:
all_names = {}

for title in titles:
    search_form = driver.find_element_by_xpath("//form[contains(@itemtype, 'http://schema.org/SearchAction')]")
    search_bar = search_form.find_element_by_xpath(".//input")
    search_button = search_form.find_element_by_xpath(".//button")
    search_bar.send_keys(title)
    search_button.click()

    names = driver.find_elements_by_xpath(".//div[contains(@class, 'tt')]")[:5]
    names = [name.text for name in names]
    all_names[title] = names

for title, results in all_names.items():
    if len(results) > 0:
        print(f"{title}: {results}")

Magic Emperor: ['Margrave’s Bastard Son was The Emperor', 'Return of the 8th Class Magician', 'Return of The Unrivaled Spear Knight']
Helmut: The Forsaken Child: ['Helmut: The Forsaken Child']
Doctor’s Rebirth: ['Doctor’s Rebirth']
Star-Embracing Swordmaster: ['Star-Embracing Swordmaster']
Survival Story of a Sword King in a Fantasy World: ['The Indomitable Martial King']
Son's Retribution: ['Son’s Retribution']
Return Of The Shattered Constellation: ['Return Of The Shattered Constellation']
Insanely Talented Player: ['Insanely-Talented Player']
I am the Strongest Boss: ['Mr Devourer, Please Act Like a Final Boss']
The Hero Returns: ['The Hero Returns', 'The Greatest Sword Hero Returns After 69420 Years', 'The Live', 'Superhuman Battlefield', 'King of the Mound']
My School Life Pretending To Be a Worthless Person: ['My School Life Pretending To Be a Worthless Person', 'Relife Player', 'Dr. Player']
The Knight King Who Returned With a God: ['The Knight King Who Returned with a God', 'I 

In [76]:
driver.close()

In [3]:
con = sqlite3.connect("comick.db")
cur = con.cursor()

for row in cur.execute("SELECT * FROM Manga"):
    print(row)

cur.close()
con.close()

('The Greatest Estate Developer', 'https://comick.io/comic/04-the-greatest-estate-developer', 147, 147, 'LHup5qVT', 'comick', 'When civil engineering student Suho Kim falls asleep reading a fantasy novel, he wakes up as a character in the book.\n\n Suho is now in the body of Lloyd Frontera, a lazy noble who loves to drink, and whose family is in a mountain of debt. Using his engineering knowledge, Suho designs inventions to avert the terrible future that lies in wait for him. \n\nWith the help of a giant hamster, a knight, and the world’s magic, can Suho dig his new family out of debt and build a better future?\n\n---\n**Links:**\n- [Official Simplified Chinese Translation](https://www.dongmanmanhua.cn/BOY/shishijilingdishejishi/list?title_no=1922)\n\n- [Official Japanese Translation](https://manga.line.me/product/periodic?id=Z0001690)')
('The Infinite Mage', 'https://comick.io/comic/01-infinite-mage', 71, 72, 'xB8MSvtf', 'comick', "This is the tale of a boy dreaming about infinity as 

In [24]:
def test(*args):
    args = list(args)
    for i, data in enumerate(args):
        if len(data) == 0:
            args[i] = ""
        elif len(data) == 1:
            args[i] = data[0]
        else:
            args[i] = ", ".join(data)
    print(args)
    return args

a = ['1', '2', '3']
b = ['3', '4', '5']
c = ['5', '6', '7']

aa = [""]
bb = [""]

aa, bb  = test(aa, bb)
print(aa, bb)

['', '']
 


In [38]:
import tkinter as tk
import ttkbootstrap as ttk

window = tk.Tk()
window.columnconfigure(0, weight=1, minsize=150)
window.columnconfigure(1, weight=1, minsize=2)
window.columnconfigure(2, weight=1, minsize=250)
window.rowconfigure(1, weight=1, minsize=2)
window.rowconfigure([0, 2], weight=1, minsize=100)

label1 = tk.Label(text="A")
label1.grid(row=0, column=0, rowspan=3, sticky='ns')

sep = ttk.Separator(window, orient='vertical')
sep.grid(row=0, column=1, rowspan=3, sticky='ns')

# label2 = tk.Label(text="B")
# label2.grid(row=1, column=0)

label1 = tk.Label(text="A")
label1.grid(row=0, column=2)

sep = ttk.Separator(window, orient='horizontal')
sep.grid(row=1, column=1, columnspan=2, sticky='ew')

label2 = tk.Label(text="B")
label2.grid(row=2, column=2)

window.mainloop()

In [39]:
import tkinter as tk

window = tk.Tk()

for i in range(3):
    window.columnconfigure(i, weight=2, minsize=75)
    window.rowconfigure(i, weight=1, minsize=50)

    for j in range(0, 3):
        frame = tk.Frame(
            master=window,
            relief=tk.RAISED,
            borderwidth=1
        )
        frame.grid(row=i, column=j, padx=5, pady=5)
        label = tk.Label(master=frame, text=f"Row {i}\nColumn {j}")
        label.pack(padx=5, pady=5)

window.mainloop()